In [7]:
import os
import tensorflow as tf

2023-12-19 13:00:05.071068: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-19 13:00:06.982346: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
os.getcwd()

'/home/tantanmanh/KLTN/Sample Model'

In [8]:
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
train_gen = img_gen.flow_from_directory('./data',class_mode="binary")

Found 24959 images belonging to 2 classes.


In [16]:
print(train_gen.next())

(array([[[[0.6117647 , 0.52156866, 0.49803925],
         [0.5764706 , 0.48627454, 0.454902  ],
         [0.5882353 , 0.5019608 , 0.45098042],
         ...,
         [0.43529415, 0.29411766, 0.23137257],
         [0.4901961 , 0.38823533, 0.25882354],
         [0.5686275 , 0.4666667 , 0.3372549 ]],

        [[0.58431375, 0.5058824 , 0.47058827],
         [0.57254905, 0.49411768, 0.45882356],
         [0.6039216 , 0.5294118 , 0.47450984],
         ...,
         [0.42352945, 0.29411766, 0.21960786],
         [0.5176471 , 0.4156863 , 0.28627452],
         [0.5882353 , 0.48627454, 0.35686275]],

        [[0.58431375, 0.5058824 , 0.47058827],
         [0.58431375, 0.5058824 , 0.46274513],
         [0.6117647 , 0.5372549 , 0.4784314 ],
         ...,
         [0.43529415, 0.3137255 , 0.23529413],
         [0.53333336, 0.43137258, 0.3019608 ],
         [0.6       , 0.49803925, 0.36862746]],

        ...,

        [[0.8235295 , 0.7294118 , 0.7137255 ],
         [0.82745105, 0.73333335, 0.7176471 

In [6]:
os.listdir(os.path.join(os.getcwd(),'data'))

['Dog', 'Cat']